In [ ]:
import os
import pandas as pd
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive')
from numpy.random import randint
from torch.utils.data import Dataset, DataLoader
import torch

Mounted at /content/drive


In [ ]:
class imuDataset(Dataset):
    def __init__(self, dataDir,xCols, yCols, windowSize = 512, transform=None, target_transform=None):
        self.xCols = xCols
        self.yCols = yCols
        self.dataDir = dataDir
        self.windowSize = windowSize
        self.transform = transform
        self.target_transform = target_transform
        self.files = dataDir

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # get filepath for random file index
        samp_path = self.files[idx]
        # load the file
        dat = pd.read_csv(samp_path, index_col='Unnamed: 0')
        # get random window in file
        starti = randint(0,dat.shape[0] - self.windowSize)
        endi = starti + self.windowSize
        dat = dat.iloc[starti:endi]
        #select variables
        x = torch.tensor(dat[self.xCols].values)
        y = torch.tensor(dat[self.yCols].values)
        
        # do optional transforms
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)

        if idx == self.__len__():
            raise IndexError            
        return x, y

In [ ]:
class imuDataset_preload(Dataset):
    def __init__(self, dataDir,xCols, yCols, windowSize = 512, transform=None, target_transform=None):
        self.xCols = xCols
        self.yCols = yCols
        self.dataDir = dataDir
        self.windowSize = windowSize
        self.transform = transform
        self.target_transform = target_transform
        self.files = dataDir
        # pre-load all files
        dataSets = []
        for file in dataDir:
            samp_path = file
            # load the file
            dat = pd.read_csv(samp_path, index_col='Unnamed: 0')            
            dataSets.append(dat)
        self.dataSets = dataSets

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # get the file
        dat = self.dataSets[idx]
        # get random window in file
        starti = randint(0,dat.shape[0] - self.windowSize)
        endi = starti + self.windowSize
        dat = dat.iloc[starti:endi]
        #select variables
        x = torch.tensor(dat[self.xCols].values)
        y = torch.tensor(dat[self.yCols].values)
        
        # do optional transforms
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)

        if idx == self.__len__():
            raise IndexError            
        return x, y

In [ ]:
base_path = '/content/drive/MyDrive/Neuromatch/'
out_path = base_path + 'preproc/'

# list of file paths
files = [str(file) for file in list(Path(out_path).glob('*'))]

input = ['RightForeArm_rot_0',
         'RightForeArm_rot_1',
         'RightForeArm_rot_2',
         'RightForeArm_rot_3',
         'RightForeArm_rot_4',
         'RightForeArm_rot_5',
         'RightForeArm_rot_6',
         'RightForeArm_rot_7',
         'RightForeArm_rot_8',   
         'RightForeArm_acc_0',
         'RightForeArm_acc_1',
         'RightForeArm_acc_2',]

output =    ['RightShoulder_rot_0',
            'RightShoulder_rot_1',
            'RightShoulder_rot_2',
            'RightShoulder_rot_3',
            'RightShoulder_rot_4',
            'RightShoulder_rot_5',
            'RightShoulder_rot_6',
            'RightShoulder_rot_7',
            'RightShoulder_rot_8',

            'RightArm_rot_0',
            'RightArm_rot_1',
            'RightArm_rot_2',
            'RightArm_rot_3',
            'RightArm_rot_4',
            'RightArm_rot_5',
            'RightArm_rot_6',
            'RightArm_rot_7',
            'RightArm_rot_8',

            'RightHand_rot_0',
            'RightHand_rot_1',
            'RightHand_rot_2',
            'RightHand_rot_3',
            'RightHand_rot_4',
            'RightHand_rot_5',
            'RightHand_rot_6',
            'RightHand_rot_7',
            'RightHand_rot_8',

            'RightForeArm_acc_0',
            'RightForeArm_acc_1',
            'RightForeArm_acc_2']

In [ ]:
datset = imuDataset(files,input, output)
datLoader = DataLoader(datset, batch_size=6, shuffle = True)

In [ ]:
datset_pre = imuDataset_preload(files,input, output)
dataLoader_pre = DataLoader(datset_pre, batch_size=2, shuffle = True)

In [ ]:
for epoch in range(10):
    xBatch,yBatch = next(iter(datLoader))
    print(xBatch.shape, yBatch.shape)

torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])


In [ ]:
for epoch in range(10):
    xBatch,yBatch = next(iter(dataLoader_pre))
    print(xBatch.shape, yBatch.shape)

torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])
torch.Size([6, 512, 12]) torch.Size([6, 512, 30])


In [ ]:
for i,(xBatch,yBatch) in enumerate(dataLoader_pre):
    #xBatch,yBatch = next(iter(dataLoader_pre))
    print(i,': ',xBatch.shape, yBatch.shape)

0 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
1 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
2 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
3 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
4 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
5 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
6 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
7 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
8 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
9 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
10 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
11 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
12 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
13 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
14 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
15 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
16 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
17 :  torch.Size([2, 512, 12]) torch.Size([2, 512, 30])
18

In [ ]:
348/2

174.0